In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

In [ ]:
import pickle
from google.colab import drive

drive.mount('/content/drive')

with open('/content/drive/My Drive/completedata.pkl', 'rb') as f:
  data = pickle.load(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(data)

Data(x=[66914, 5], edge_index=[2, 537063], edge_attr=[537063, 1], y=[66914])


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

# Assuming you have data for each month in data_list
# Let's assume data_list is a list of Torch Geometric Data objects for each month

class GraphSageModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSageModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return x

# Assuming your data_list contains data for each month separately
# Each data entry should have attributes x (node features) and edge_index (adjacency list)
# Other attributes can be added as needed

# Define your model
model = GraphSageModel(in_channels=5,
                       hidden_channels=32,
                       out_channels=64)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Assuming data_list is a list of data objects
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    target = data.y.long()
    loss = criterion(out, target)
    loss.backward()
    optimizer.step()

    # After training, set model to evaluation mode
model.eval()
with torch.no_grad():
  # Perform forward pass to obtain node embeddings
  embeddings = model(data.x, data.edge_index)
  # Now, 'embeddings' contains the embeddings for each node in the current graph
  #You can store or process these embeddings as needed
print(embeddings)

tensor([[ 5934.2422,  4081.2039, -4237.2236,  ..., -4243.5776, -5677.2437,
         -1158.0588],
        [  252.4030,   170.5021,  -204.7993,  ...,  -125.5616,  -237.8200,
           -43.0297],
        [   22.0904,    13.8771,   -20.8223,  ...,   -26.7569,   -38.7667,
           -19.8298],
        ...,
        [   49.9314,    34.0621,   -38.3052,  ...,   -34.2858,   -52.5085,
           -14.2375],
        [   30.8513,    19.2386,   -22.8826,  ...,   -19.4175,   -30.7607,
            -9.0138],
        [ 7900.6533,  5338.7632, -5323.3774,  ..., -5516.3022, -7086.1870,
         -1355.6252]])


In [ ]:
Y = data.y

In [ ]:
# prompt: find the count fo 0 and 1 in Y

# Count the number of occurrences of 0 and 1 in Y
count_0 = (Y == 0).sum()
count_1 = (Y == 1).sum()

# Print the counts
print(f"Count of 0: {count_0}")
print(f"Count of 1: {count_1}")


Count of 0: 65761
Count of 1: 1153


In [ ]:
# L2 normalization

embeddings_norm = F.normalize(embeddings,p=2, dim=1)
embeddings_norm

tensor([[ 0.1871,  0.1286, -0.1336,  ..., -0.1338, -0.1790, -0.0365],
        [ 0.1866,  0.1261, -0.1514,  ..., -0.0928, -0.1758, -0.0318],
        [ 0.0999,  0.0627, -0.0941,  ..., -0.1210, -0.1753, -0.0897],
        ...,
        [ 0.1724,  0.1176, -0.1323,  ..., -0.1184, -0.1813, -0.0492],
        [ 0.1793,  0.1118, -0.1330,  ..., -0.1129, -0.1788, -0.0524],
        [ 0.1985,  0.1341, -0.1337,  ..., -0.1386, -0.1780, -0.0341]])

In [ ]:
!pip install imbalanced_learn

In [ ]:
minor = []
major = []
for i in range(len(Y)):
  if Y[i] == 1:
    minor.append(embeddings_norm[i])
  else:
    major.append(embeddings_norm[i])

In [ ]:
minor

[tensor([ 0.1978,  0.1333, -0.1335, -0.1559, -0.1501, -0.1707, -0.0927, -0.1743,
         -0.1566, -0.1035, -0.0986, -0.0572, -0.1527, -0.1281, -0.1172, -0.1026,
         -0.1066, -0.0697, -0.1663, -0.1778, -0.1376, -0.0440, -0.0705, -0.1650,
         -0.0681, -0.1177, -0.1193, -0.1483, -0.0856, -0.1304, -0.0918, -0.0629,
         -0.1083, -0.0893, -0.0699, -0.0974, -0.0944, -0.1920, -0.0358, -0.0929,
         -0.0783, -0.1186, -0.0617, -0.1317, -0.1288, -0.0472, -0.0921, -0.1906,
         -0.0753, -0.1539, -0.1359, -0.1105, -0.1477, -0.0760, -0.1995, -0.0769,
         -0.1267, -0.2255, -0.1503, -0.0447, -0.0803, -0.1384, -0.1779, -0.0345]),
 tensor([ 0.2021,  0.1353, -0.1337, -0.1562, -0.1467, -0.1695, -0.0943, -0.1746,
         -0.1560, -0.0997, -0.0960, -0.0546, -0.1529, -0.1290, -0.1191, -0.1030,
         -0.1039, -0.0686, -0.1673, -0.1781, -0.1378, -0.0410, -0.0716, -0.1656,
         -0.0697, -0.1182, -0.1189, -0.1506, -0.0869, -0.1309, -0.0928, -0.0616,
         -0.1085, -0.0892,

In [ ]:
major

[tensor([ 0.1871,  0.1286, -0.1336, -0.1537, -0.1583, -0.1746, -0.0881, -0.1750,
         -0.1560, -0.1098, -0.1047, -0.0658, -0.1518, -0.1293, -0.1086, -0.1035,
         -0.1109, -0.0696, -0.1632, -0.1766, -0.1373, -0.0479, -0.0678, -0.1638,
         -0.0649, -0.1159, -0.1207, -0.1420, -0.0840, -0.1303, -0.0901, -0.0672,
         -0.1091, -0.0902, -0.0778, -0.1005, -0.0928, -0.1949, -0.0425, -0.0969,
         -0.0898, -0.1198, -0.0687, -0.1384, -0.1262, -0.0495, -0.0839, -0.1839,
         -0.0809, -0.1611, -0.1276, -0.1094, -0.1466, -0.0794, -0.2003, -0.0765,
         -0.1305, -0.2261, -0.1464, -0.0317, -0.0821, -0.1338, -0.1790, -0.0365]),
 tensor([ 0.1866,  0.1261, -0.1514, -0.1408, -0.1816, -0.1648, -0.0561, -0.1577,
         -0.1495, -0.1319, -0.1559, -0.0679, -0.1511, -0.0905, -0.1387, -0.0769,
         -0.1263, -0.0906, -0.1663, -0.1825, -0.1354, -0.0774, -0.0535, -0.1592,
         -0.0456, -0.1600, -0.1301, -0.1137, -0.0609, -0.1214, -0.0764, -0.0758,
         -0.1048, -0.0992,

In [ ]:
from imblearn.over_sampling import SMOTE
import numpy as np

In [ ]:
smote = SMOTE()
X_res, y_res = smote.fit_resample(embeddings_norm, Y)

In [ ]:
# prompt: find the count fo 0 and 1 in Y

# Count the number of occurrences of 0 and 1 in Y
count_0 = (y_res == 0).sum()
count_1 = (y_res == 1).sum()

# Print the counts
print(f"Count of 0: {count_0}")
print(f"Count of 1: {count_1}")

Count of 0: 65761
Count of 1: 65761


In [ ]:
# prompt: for x_res and y_res classify them using SVM also import train test split

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

# Create an SVM classifier
clf = SVC(kernel='linear')

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = clf.predict(X_test)

# Print the accuracy of the classifier



NameError: name 'accuracy_score' is not defined

In [ ]:
# prompt: make classification report using scikitlearn for ypred and ytest

from sklearn.metrics import classification_report

# Get the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


              precision    recall  f1-score   support

         0.0       0.84      0.67      0.75     19792
         1.0       0.72      0.87      0.79     19665

    accuracy                           0.77     39457
   macro avg       0.78      0.77      0.77     39457
weighted avg       0.78      0.77      0.77     39457



In [ ]:
# prompt: for x_res and y_res classify them using knn also import train test split

from sklearn.neighbors import KNeighborsClassifier

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

# Create a KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the classifier
knn.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = knn.predict(X_test)

# Get the accuracy of the classifier
accuracy = knn.score(X_test, y_test)

# Print the accuracy
print(f"Accuracy: {accuracy}")

# Get the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


Accuracy: 0.9587145500164737
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96     19792
         1.0       0.93      0.99      0.96     19665

    accuracy                           0.96     39457
   macro avg       0.96      0.96      0.96     39457
weighted avg       0.96      0.96      0.96     39457



In [ ]:
# prompt: for x_res and y_res classify them using randomforest also import train test split

from sklearn.ensemble import RandomForestClassifier

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

# Create a Random Forest classifier
rfc = RandomForestClassifier(n_estimators=100)

# Train the classifier
rfc.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = rfc.predict(X_test)

# Get the accuracy of the classifier
accuracy = rfc.score(X_test, y_test)

# Print the accuracy
print(f"Accuracy: {accuracy}")

# Get the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


Accuracy: 0.9791671946676128
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     19792
         1.0       0.97      0.99      0.98     19665

    accuracy                           0.98     39457
   macro avg       0.98      0.98      0.98     39457
weighted avg       0.98      0.98      0.98     39457



In [ ]:
# prompt: save the model

# Save the model to a file
torch.save(model.state_dict(), '/content/drive/My Drive/embeddmodel.pt')
